In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.models import  Model
import numpy as np
import tensorflow_datasets as tfds

# from data.id_dataloader import load_cifar10, load_intel_image, load_mnist, load_cifar100
from data.classes import cifar10_classes, mnist_classes, intel_image_classes, cifar100_classes

from models.models import resnet50, wideresnet2810, vgg16, inceptionv3, efficientnetb2
from models.pretrained_models import pretrained_resnet50, pretrained_vgg16

from rsnn_functions.budgeting import train_embeddings, fit_gmm, ellipse, overlaps
from rsnn_functions.bf_encoding_gt import groundtruthmod
from rsnn_functions.belief_mass_betp import belief_to_mass, mass_coeff, final_betp
from rsnn_functions.rsnn_loss import BinaryCrossEntropy

from utils.train_utils import lr_schedule, train_val_split, data_generator, lr_callbacks, save_model_and_weights

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    num_gpus = len(gpus)
    print(f"Number of GPUs available: {num_gpus}")
    # Set GPUs to use. For example, limit TensorFlow to use 3 GPUs
    tf.config.experimental.set_visible_devices(gpus[:3], 'GPU')
    
# Create a MirroredStrategy for multi-GPU use
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of GPUs available: 1
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1


In [3]:
# Initializing parameters 
k = 20  #number of number of non-singleton focal sets 
batch_size = 128
epochs = 2

In [4]:
num_classes = {"cifar10": 10, "mnist": 10, "intel_image": 6, "cifar100": 100, "svhn": 10, "fmnist": 10, "kmnist":10}

# dataset_loader = {
#  "cifar10": load_cifar10, 
#  "mnist": load_mnist, 
#  "intel_image": load_intel_image, 
#  "cifar100": load_cifar100, 
# }

models = {
    "resnet50": resnet50, 
    "wideresnet_28_10": wideresnet2810, 
    "vgg16": vgg16,
    "inception_v3": inceptionv3,
    "efficientnet_b2": efficientnetb2
}

pretrained_models = {
    "pretrained_resnet50": pretrained_resnet50, 
    "pretrained_vgg16": pretrained_vgg16,
}

class_list_functions = {
    "cifar10": cifar10_classes,
     "mnist": mnist_classes, 
    "intel_image": intel_image_classes, 
    "cifar100": cifar100_classes, 
}

In [5]:
# # Define configurations
# selected_dataset = "cifar10"  # Choose the dataset
# selected_model = "resnet50"   # Choose the model
# batch_size = 128
# epochs = 100

# # Class list
# classes = class_list_functions[selected_dataset]()
# print("Classes:", classes)

# num_clusters = len(classes)
# classes_dict = {c:num for c,num in zip(classes, range(len(classes)))}
# classes_dict_inverse = {num:c for c,num in zip(classes, range(len(classes)))}

# # Load dataset based on selected_dataset
# x_train, y_train, x_test_org, x_test, y_test = dataset_loader[selected_dataset]()

# # Infer input_shape based on selected_dataset
# input_shape = x_train.shape[1:]

# # Train-validation split
# x_train, y_train, y_train_one_hot, x_val, y_val, y_val_one_hot = train_val_split(x_train, y_train, num_classes[selected_dataset], val_samples=-10000)

# print("Shape of x_train:", x_train.shape)
# print("Shape of x_test:", x_test.shape)
# print("Shape of x_val:", x_val.shape)

# # Learning rate scheduler
# callbacks = lr_callbacks(lr_schedule)

# # Data augmentation
# datagen = data_generator(x_train)

# ---- New Data Loading and Preparation using tensorflow-datasets ----

# ---- Define necessary configuration variables ----
selected_dataset = "cifar10"
selected_model = "wideresnet2810"
IMG_SIZE = 32 # CIFAR-10 image size (already defined later, but good to have here)
input_shape = (IMG_SIZE, IMG_SIZE, 3) # Define input shape for the model
# --- Add these lines ---
if selected_dataset == 'cifar10':
    input_shape = (32, 32, 3)
    num_classes = {'cifar10': 10} # Assuming only CIFAR-10 for now
elif selected_dataset == 'mnist':
    input_shape = (28, 28, 1) # MNIST is grayscale
    num_classes = {'mnist': 10}
# Add elif blocks for other datasets if you plan to use them (e.g., cifar100, intel_image)
else:
    raise ValueError(f"Dataset {selected_dataset} not configured.")
# --- End of lines to add ---

# You should already have these lines below in the same cell:
# Dictionary mapping model names to functions (example)
pretrained_models = {
    'resnet50': resnet50,
    'vgg16': vgg16,
    'inceptionv3': inceptionv3,
    'efficientnetb2': efficientnetb2,
}

models = {
     'wideresnet2810': wideresnet2810,
}

print(f"Selected Dataset: {selected_dataset}")
print(f"Selected Model: {selected_model}")
print(f"Input Shape: {input_shape}")
print(f"Number of Classes: {num_classes[selected_dataset]}")

# -------------------------------------------------

# Load CIFAR-10 dataset info and the dataset itself
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train[:80%]', 'train[80%:]', 'test'], # Split train into 80% train, 20% validation
    shuffle_files=True,
    as_supervised=True, # Return (image, label) tuples
    with_info=True,
)

# Get number of classes and examples
num_classes_from_ds = ds_info.features['label'].num_classes
num_train_examples = ds_info.splits['train[:80%]'].num_examples
num_val_examples = ds_info.splits['train[80%:]'].num_examples
num_test_examples = ds_info.splits['test'].num_examples
print(f"Num Train: {num_train_examples}, Num Val: {num_val_examples}, Num Test: {num_test_examples}, Num Classes: {num_classes_from_ds}")

BATCH_SIZE = 128 # Same as before, adjust if needed
callbacks = lr_callbacks(lr_schedule)

# Preprocessing function
def preprocess_image(image, label):
    image = tf.cast(image, tf.float32)
    # Normalize images to the range [0, 1]
    image = image / 255.0
    # Optional: Add data augmentation here if desired
    # image = tf.image.random_flip_left_right(image)
    # ... other augmentations ...
    label = tf.one_hot(label, depth=num_classes_from_ds) # Use num_classes_from_ds or the actual number of classes (e.g., 10 for CIFAR-10)

    return image, label

# Apply preprocessing and batching
ds_train = ds_train.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache() # Cache data after first epoch for performance
ds_train = ds_train.shuffle(num_train_examples // 10) # Shuffle buffer size
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE) # Prefetch batches

ds_val = ds_val.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE)
ds_val = ds_val.cache()
ds_val = ds_val.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

print("Data loading and preprocessing complete using tfds.")
# ---- End of new Data Loading ----

Selected Dataset: cifar10
Selected Model: wideresnet2810
Input Shape: (32, 32, 3)
Number of Classes: 10
Num Train: 40000, Num Val: 10000, Num Test: 10000, Num Classes: 10
Data loading and preprocessing complete using tfds.


## CNN

In [6]:
# Multi-GPU run
with strategy.scope():      
    # Create the model based on selected_model
    if selected_model in pretrained_models:
        model = pretrained_models[selected_model](input_shape=input_shape,  num_classes=num_classes[selected_dataset], final_activation='softmax')
    else:
        model = models[selected_model](input_shape=input_shape, num_classes=num_classes[selected_dataset], final_activation='softmax')

    # Compile the model 
    model.compile(loss='categorical_crossentropy',
                optimizer="adam",
                metrics=['accuracy'])

model.summary()

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 32, 32, 96)           2688      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 32, 32, 96)           384       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 re_lu (ReLU)                (None, 32, 32, 96)           0         ['batch_normalization[0][0

In [7]:
# ---- Original model.fit call (commented out) ----
# history = model.fit(datagen.flow(x_train, y_train_one_hot, batch_size=batch_size),
#                     validation_data=(x_val, y_val_one_hot),
#                     epochs=epochs, verbose=1, workers=2)
# ---- End of original call ----

# ---- Updated model.fit using tf.data datasets ----

print("Starting model training...")
history = model.fit(
    ds_train,
    epochs=epochs,
    validation_data=ds_val,
    # Calculate steps per epoch explicitly if using tf.data with infinite repeat
    # steps_per_epoch=num_train_examples // BATCH_SIZE,
    # validation_steps=num_val_examples // BATCH_SIZE,
    callbacks=callbacks # Use the same callbacks as before
)
print("Model training finished.")

Starting model training...
Learning rate:  0.001
Epoch 1/2


2025-05-09 19:17:23.672607: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:551] The `assert_cardinality` transformation is currently not handled by the auto-shard rewrite and will be removed.


313/313 [==============================] - ETA: 0s - loss: 2.0706 - accuracy: 0.4212 

2025-05-09 22:09:12.556199: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:551] The `assert_cardinality` transformation is currently not handled by the auto-shard rewrite and will be removed.


313/313 [==============================] - 11202s 36s/step - loss: 2.0706 - accuracy: 0.4212 - val_loss: 2.5706 - val_accuracy: 0.2435 - lr: 0.0010
Learning rate:  0.001
Epoch 2/2
313/313 [==============================] - 10339s 33s/step - loss: 1.4664 - accuracy: 0.5378 - val_loss: 1.7990 - val_accuracy: 0.4488 - lr: 0.0010
Model training finished.


In [8]:
# Save model and weights
save_model_and_weights(model, selected_model, selected_dataset, model_type='CNN')

## BUDGETING

In [9]:
print("Preparing data for budgeting steps...")

# 1. Get class names
classes = ds_info.features['label'].names
print(f"Classes: {classes}")

# 2. num_clusters
num_clusters = len(classes) # This is the same as num_classes_from_ds
print(f"Number of clusters/classes: {num_clusters}")

# 3. classes_dict
classes_dict = {name: i for i, name in enumerate(classes)}
print(f"Classes Dict: {classes_dict}")

# 4. classes_dict_inverse
classes_dict_inverse = {i: name for i, name in enumerate(classes)}
print(f"Classes Dict Inverse: {classes_dict_inverse}")

# 5. Reconstruct x_train, y_train as NumPy arrays from the original training split
# We'll take the full training set, preprocess it, but not batch it for this purpose.
# This assumes 'train' was split into 'train[:80%]' and 'train[80%:]' earlier.
# Let's load the original full 'train' split from tfds.
full_train_dataset = tfds.load('cifar10', split='train', as_supervised=True)

# Preprocess all of it (normalize images, but keep integer labels for y_train for now)
def preprocess_for_numpy(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

full_train_dataset_processed = full_train_dataset.map(preprocess_for_numpy, num_parallel_calls=tf.data.AUTOTUNE)

# Convert to NumPy arrays
# This will load all training images into memory.
train_images_list = []
train_labels_list = []
for image, label in tfds.as_numpy(full_train_dataset_processed):
    train_images_list.append(image)
    train_labels_list.append(label)

x_train_budget = np.array(train_images_list)
y_train_budget = np.array(train_labels_list) # These are integer labels

print(f"x_train_budget shape: {x_train_budget.shape}")
print(f"y_train_budget shape: {y_train_budget.shape}")
print(f"Unique y_train_budget labels: {np.unique(y_train_budget)}")


# For the RS-NN part later, you'll also need x_val, y_val, x_test, y_test
# We can prepare them similarly if you confirm they are needed for the groundtruthmod function

# For example, for x_val, y_val:
val_dataset = tfds.load('cifar10', split='train[80%:]', as_supervised=True) # This was ds_val's original source
val_dataset_processed = val_dataset.map(preprocess_for_numpy, num_parallel_calls=tf.data.AUTOTUNE)
val_images_list = []
val_labels_list = []
for image, label in tfds.as_numpy(val_dataset_processed):
    val_images_list.append(image)
    val_labels_list.append(label)
x_val_budget = np.array(val_images_list)
y_val_budget = np.array(val_labels_list)
print(f"x_val_budget shape: {x_val_budget.shape}")
print(f"y_val_budget shape: {y_val_budget.shape}")

# And for x_test, y_test:
test_dataset = tfds.load('cifar10', split='test', as_supervised=True) # This was ds_test's original source
test_dataset_processed = test_dataset.map(preprocess_for_numpy, num_parallel_calls=tf.data.AUTOTUNE)
test_images_list = []
test_labels_list = []
for image, label in tfds.as_numpy(test_dataset_processed):
    test_images_list.append(image)
    test_labels_list.append(label)
x_test_budget = np.array(test_images_list)
y_test_budget = np.array(test_labels_list)
print(f"x_test_budget shape: {x_test_budget.shape}")
print(f"y_test_budget shape: {y_test_budget.shape}")


print("Data preparation for budgeting complete.")


Preparing data for budgeting steps...
Classes: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
Number of clusters/classes: 10
Classes Dict: {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}
Classes Dict Inverse: {0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}
x_train_budget shape: (50000, 32, 32, 3)
y_train_budget shape: (50000,)
Unique y_train_budget labels: [0 1 2 3 4 5 6 7 8 9]
x_val_budget shape: (10000, 32, 32, 3)
y_val_budget shape: (10000,)
x_test_budget shape: (10000, 32, 32, 3)
y_test_budget shape: (10000,)
Data preparation for budgeting complete.


In [10]:
# Extracting features from the penultimate layer
aux_model = Model(model.input, model.layers[-2].output)

# 3D feature space respresentation of class embeddings
train_embedded_tsne = train_embeddings(aux_model, x_train_budget, batch_size)

# Fitting Gaussian Mixture Models (GMM) to individual classes
individual_gms = fit_gmm(classes, train_embedded_tsne, y_train_budget)

# Calculating clusters for each class
regions, means, max_len = ellipse(individual_gms, num_classes[selected_dataset])

# Compute the overlap and choose the sets of classes with highest overlap
new_classes = overlaps(k, classes, num_clusters, classes_dict, regions, means, max_len)

# np.save('new_classes.npy', new_classes)
print(new_classes)

391/391 [==============================] - 6897s 18s/step


python(79058) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


[{'airplane'}, {'automobile'}, {'bird'}, {'cat'}, {'deer'}, {'dog'}, {'frog'}, {'horse'}, {'ship'}, {'truck'}, {'deer', 'bird'}, {'dog', 'cat'}, {'deer', 'frog'}, {'airplane', 'ship'}, {'truck', 'automobile'}, {'cat', 'deer'}, {'horse', 'bird'}, {'frog', 'bird'}, {'cat', 'bird'}, {'horse', 'cat'}, {'dog', 'deer'}, {'dog', 'bird'}, {'truck', 'ship'}, {'dog', 'horse'}, {'frog', 'deer', 'bird'}, {'horse', 'deer'}, {'cat', 'deer', 'bird'}, {'dog', 'cat', 'deer'}, {'ship', 'automobile'}, {'horse', 'frog'}]


In [11]:
## Load saved new_classes
# new_classes = np.load('new_classes.npy', allow_pickle=True)

In [12]:
# Belief-encoding of the ground truth
y_train_modified = groundtruthmod(y_train_budget, classes, new_classes, classes_dict_inverse)
y_val_modified = groundtruthmod(y_val_budget, classes, new_classes, classes_dict_inverse)
y_test_modified = groundtruthmod(y_test_budget, classes, new_classes, classes_dict_inverse)

## RS-NN

In [13]:
# Multi-GPU run
with strategy.scope():      
    # Create the model based on selected_model
    if selected_model in pretrained_models:
        new_model = pretrained_models[selected_model](input_shape=input_shape,  num_classes=len(new_classes), final_activation='sigmoid')
    else:
        new_model = models[selected_model](input_shape=input_shape, num_classes=len(new_classes), final_activation='sigmoid')

    # Compile the model 
    new_model.compile(loss=BinaryCrossEntropy,
                optimizer="adam",
                metrics=['binary_accuracy'])

new_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d_27 (Conv2D)          (None, 32, 32, 96)           2688      ['input_2[0][0]']             
                                                                                                  
 batch_normalization_27 (Ba  (None, 32, 32, 96)           384       ['conv2d_27[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 re_lu_13 (ReLU)             (None, 32, 32, 96)           0         ['batch_normalization_27

In [14]:
# history_new = new_model.fit(datagen.flow(x_train, y_train_modified, batch_size=batch_size),
#                     validation_data=(x_val, y_val_modified),
#                     epochs=epochs, verbose=1, workers=2,
#                    callbacks=callbacks)

In [15]:
    print("Preparing datasets for RS-NN training...")

    # y_train_modified, y_val_modified, y_test_modified should be defined from the budgeting/belief-encoding section
    # x_train_budget, x_val_budget, x_test_budget are also defined

    # Ensure modified labels are float32 for Keras BinaryCrossEntropy loss
    y_train_modified_float = tf.cast(y_train_modified, tf.float32)
    y_val_modified_float = tf.cast(y_val_modified, tf.float32)
    # y_test_modified_float = tf.cast(y_test_modified, tf.float32) # If you create a ds_test_rsnn

    print(f"y_train_modified shape: {y_train_modified_float.shape}, dtype: {y_train_modified_float.dtype}")
    print(f"y_val_modified shape: {y_val_modified_float.shape}, dtype: {y_val_modified_float.dtype}")

    # Create tf.data.Dataset objects
    # We'll use x_train_budget (images) and y_train_modified_float (new labels)
    ds_train_rsnn = tf.data.Dataset.from_tensor_slices((x_train_budget, y_train_modified_float))
    ds_val_rsnn = tf.data.Dataset.from_tensor_slices((x_val_budget, y_val_modified_float))

    # Apply batching and prefetching
    # batch_size should be defined from an earlier cell (e.g., batch_size = 128)
    ds_train_rsnn = ds_train_rsnn.shuffle(len(x_train_budget) // 10) # Shuffle the data
    ds_train_rsnn = ds_train_rsnn.batch(batch_size)
    ds_train_rsnn = ds_train_rsnn.prefetch(tf.data.AUTOTUNE)

    ds_val_rsnn = ds_val_rsnn.batch(batch_size)
    ds_val_rsnn = ds_val_rsnn.prefetch(tf.data.AUTOTUNE)

    # If you plan to evaluate on the test set with new_model:
    # y_test_modified_float = tf.cast(y_test_modified, tf.float32)
    # ds_test_rsnn = tf.data.Dataset.from_tensor_slices((x_test_budget, y_test_modified_float))
    # ds_test_rsnn = ds_test_rsnn.batch(batch_size)
    # ds_test_rsnn = ds_test_rsnn.prefetch(tf.data.AUTOTUNE)
    # print(f"ds_test_rsnn element spec: {ds_test_rsnn.element_spec if 'ds_test_rsnn' in locals() else 'Not created'}")


    print("RS-NN datasets prepared.")
    print(f"ds_train_rsnn element spec: {ds_train_rsnn.element_spec}")
    print(f"ds_val_rsnn element spec: {ds_val_rsnn.element_spec}")

Preparing datasets for RS-NN training...
y_train_modified shape: (50000, 30), dtype: <dtype: 'float32'>
y_val_modified shape: (10000, 30), dtype: <dtype: 'float32'>
RS-NN datasets prepared.
ds_train_rsnn element spec: (TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 30), dtype=tf.float32, name=None))
ds_val_rsnn element spec: (TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 30), dtype=tf.float32, name=None))


In [16]:
history_new = new_model.fit(
    ds_train_rsnn,  # Use the new dataset
    validation_data=ds_val_rsnn, # Use the new validation dataset
    epochs=epochs,
    verbose=1,
    callbacks=callbacks # callbacks should still be defined from earlier
    # workers parameter is not typically used with tf.data datasets in this way
)

2025-05-10 05:46:03.568962: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 50000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\026TensorSliceDataset:148"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 30
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
exp

Learning rate:  0.001
Epoch 1/2
391/391 [==============================] - ETA: 0s - loss: 0.3607 - binary_accuracy: 0.8523 

2025-05-10 11:05:32.910880: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:786] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_FLOAT
      type: DT_FLOAT
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 10000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\026TensorSliceDataset:149"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 32
        }
        dim {
          size: 32
        }
        dim {
          size: 3
        }
      }
      shape {
        dim {
          size: 30
        }
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
exp

391/391 [==============================] - 21133s 54s/step - loss: 0.3607 - binary_accuracy: 0.8523 - val_loss: 0.3889 - val_binary_accuracy: 0.8407 - lr: 0.0010
Learning rate:  0.001
Epoch 2/2
391/391 [==============================] - 16655s 43s/step - loss: 0.2866 - binary_accuracy: 0.8818 - val_loss: 0.9933 - val_binary_accuracy: 0.8450 - lr: 0.0010


In [17]:
# history_new = new_model.fit(datagen.flow(x_train, y_train_modified, batch_size=batch_size),
#                     validation_data=(x_val, y_val_modified),
#                     epochs=epochs, verbose=1, workers=2,
#                    callbacks=callbacks)

In [18]:
# # Save model and weights
save_model_and_weights(new_model, selected_model, selected_dataset, model_type='RSNN')